In [1]:
import os
import sys
#path environment
if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:
    sys.exit("please declare environment variable 'SUMO_HOME'")
import traci

import xml.etree.ElementTree as ET

import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
import statsmodels.api as sm

import sumolib

import heapq


In [2]:
file_output = "my_output_file_grid_min.xml"

sumo_cmd = ['sumo', 
            '--duration-log.statistics',
            '--tripinfo-output', file_output, 
            '-c', 'maps/grid/grid.sumo.cfg']

In [3]:
def analyze_time_loss(file_path):
    # Parse the XML file
    tree = ET.parse(file_path)
    root = tree.getroot()

    # Initialize variables for analysis
    total_time_loss = 0.0
    total_waiting_time = 0.0
    total_duration = 0.0
    total_reroutes = 0
    vehicle_count = 0

    # Iterate over each tripinfo element in the XML
    for trip in root.findall('tripinfo'):
        time_loss = float(trip.get('timeLoss'))
        waiting_time = float(trip.get('waitingTime'))
        duration = float(trip.get('duration'))
        reroutes = int(trip.get('rerouteNo'))

        total_time_loss += time_loss
        total_waiting_time += waiting_time
        total_duration += duration
        total_reroutes += reroutes
        vehicle_count += 1

    # Calculate averages
    average_time_loss = total_time_loss / vehicle_count if vehicle_count > 0 else 0
    average_waiting_time = total_waiting_time / vehicle_count if vehicle_count > 0 else 0
    average_duration = total_duration / vehicle_count if vehicle_count > 0 else 0
    average_reroutes = total_reroutes / vehicle_count if vehicle_count > 0 else 0

    # Store results in a dictionary
    results = {
        'Total Vehicles': vehicle_count,
        'Average Time Loss': average_time_loss,
        'Average Waiting Time': average_waiting_time,
        'Average Duration': average_duration,
        'Total Reroutes': total_reroutes,
        'Average Reroutes': average_reroutes
    }
    return results


In [4]:
net = sumolib.net.readNet('maps/grid/grid.net.xml') # Atenção: Adicionar o .net para construir o grafo

graph = {}  # Armazenar o grafo de edges e seus pesos

def build_graph():
    global graph
    for edge in net.getEdges():
        edge_id = edge.getID()

        # Ignorar vias internas que começam com ":"
        if edge_id.startswith(":"):
            continue

        graph[edge_id] = {
            'from': edge.getFromNode().getID(),
            'to': edge.getToNode().getID(),
            'length': edge.getLength(),
            'current_travel_time': edge.getLength() / edge.getSpeed(),
        }


In [5]:
# Função para atualizar os pesos do grafo em tempo real
def update_edge_weights():
    global graph
    for edge_id in graph:
        # num_vehicles = traci.edge.getLastStepVehicleNumber(edge_id)
        mean_speed = traci.edge.getLastStepMeanSpeed(edge_id)

        if mean_speed > 0:
            # Calcula o tempo de viagem atual com base na velocidade média dos veículos
            travel_time = graph[edge_id]['length'] / mean_speed
        else:
            # Penalizar vias com veículos parados (velocidade 0)
            travel_time = float('inf')

        # Atualizar o tempo de viagem no grafo
        graph[edge_id]['current_travel_time'] = travel_time

In [6]:

# Algoritmo de Dijkstra para encontrar o caminho mínimo
def dijkstra(start_edge, end_edge):
    global graph
    queue = []
    heapq.heappush(queue, (0, start_edge))  # (peso acumulado, aresta atual)
    distances = {edge: float('inf') for edge in graph}
    distances[start_edge] = 0
    predecessors = {}

    while queue:
        current_distance, current_edge = heapq.heappop(queue)

        if current_edge == end_edge:
            break

        for neighbor in net.getEdge(current_edge).getOutgoing():
            neighbor_id = neighbor.getID()
            weight = graph[neighbor_id]['current_travel_time']
            distance = current_distance + weight

            if distance < distances[neighbor_id]:
                distances[neighbor_id] = distance
                predecessors[neighbor_id] = current_edge
                heapq.heappush(queue, (distance, neighbor_id))

    # Reconstruir o caminho
    path = []
    step = end_edge
    while step in predecessors:
        path.insert(0, step)
        step = predecessors[step]
    if path:
        path.insert(0, start_edge)
    return path


In [7]:
def calculate_congestion_threshold(edge):
    """
    Define o tempo máximo de viagem permitido para considerar uma via como congestionada.
    Pode ser um múltiplo do tempo ideal de viagem (baseado na velocidade máxima permitida).
    """
    ideal_travel_time = edge.getLength() / edge.getSpeed()  # Tempo ideal (sem congestionamento)
    congestion_threshold = ideal_travel_time * 1.2
    return congestion_threshold

def is_route_congested(vehicle_id):
    """
    Verifica se a rota futura de um veículo contém vias congestionadas.
    Usa o `congestion_threshold` para identificar congestionamento.
    """
    route_edges = traci.vehicle.getRoute(vehicle_id)

    for edge_id in route_edges:
        if edge_id in graph:
            current_travel_time = graph[edge_id]['current_travel_time']

            # Calcular o limiar de congestionamento para essa via
            congestion_threshold = calculate_congestion_threshold(net.getEdge(edge_id))

            # Verificar se o tempo de viagem atual excede o limiar
            if current_travel_time > congestion_threshold:
                return True  # A via está congestionada
    return False


def is_current_route_optimal(vehicle_id):
    """
    Verifica se a rota atual do veículo é o caminho mínimo atualizado.
    """
    current_edge = traci.vehicle.getRoadID(vehicle_id)
    destination = traci.vehicle.getRoute(vehicle_id)[-1]

    # Calcular o caminho mínimo baseado no grafo atualizado
    optimal_route = dijkstra(current_edge, destination)

    # Comparar a rota atual com o caminho mínimo calculado
    current_route = traci.vehicle.getRoute(vehicle_id)
    return current_route == optimal_route

def reroute_if_necessary(vehicle_id):
    current_edge = traci.vehicle.getRoadID(vehicle_id)

    # Ignorar vias internas que começam com ":"
    if current_edge.startswith(":"):
        print(f"Edge {current_edge} é uma via interna, ignorando reroteamento.")
        return

    destination = traci.vehicle.getRoute(vehicle_id)[-1]

    # Atualizar os pesos e rerotear
    update_edge_weights()
    new_route = dijkstra(current_edge, destination)

    # Definir a nova rota para o veículo
    if new_route:
        traci.vehicle.setRoute(vehicle_id, new_route)
        print(f"Veículo {vehicle_id} reroteado pela nova rota {new_route}")



In [8]:
# # Rerotear um veículo com base no caminho mínimo
# def reroute_vehicle(vehicle_id):
#     current_edge = traci.vehicle.getRoadID(vehicle_id)
#     destination = traci.vehicle.getRoute(vehicle_id)[-1]  # Obter o destino final do veículo
#
#     # Ignorar vias internas que começam com ":"
#     if current_edge.startswith(":"):
#         print(f"Edge {current_edge} é uma via interna, ignorando reroteamento.")
#         return
#
#     # Atualizar os pesos com base no tráfego
#     update_edge_weights()
#
#     # Calcular o caminho mínimo com base nos pesos atualizados
#     new_route = dijkstra(current_edge, destination)
#
#     if new_route:
#         # Definir a nova rota para o veículo
#         traci.vehicle.setRoute(vehicle_id, new_route)
#         print(f"Veículo {vehicle_id} reroteado pela rota {new_route}")


In [9]:
# reroute_intervals = [5, 30, 60, 120]  # Different intervals to test
reroute_intervals = [60]
STEPTOTAL = 3600
simulation_results = []

# Construir o grafo
build_graph()

for reroute_interval in reroute_intervals:
    print(f"Running simulation with reroute interval: {reroute_interval} seconds")
    
    # Start SUMO
    traci.start(sumo_cmd)

    step = 0
    try:
        while step < STEPTOTAL:
            traci.simulationStep()
    
            if step % reroute_interval == 0:
                for vehicle_id in traci.vehicle.getIDList():
                    reroute_if_necessary(vehicle_id)
    
            step += 1
    finally:
        traci.close()


    
    # Analyze the results and store them in the simulation_results list
    results = analyze_time_loss(file_output)
    results['Reroute Interval'] = reroute_interval  # Add the interval as a data point
    simulation_results.append(results)



Running simulation with reroute interval: 60 seconds
 Retrying in 1 seconds
***Starting server on port 34449 ***
Loading net-file from 'maps/grid/grid.net.xml' ... done (37ms).
Loading done.
Simulation version 1.18.0 started with time: 0.00.
Veículo 0 reroteado pela nova rota ['I7H7', 'H7G7', 'G7F7', 'F7E7', 'E7D7', 'D7C7', 'C7B7', 'B7B6', 'B6B5', 'B5B4', 'B4B3', 'B3B2', 'B2B1', 'B1A1']
Veículo 0 reroteado pela nova rota ['F7E7', 'E7D7', 'D7C7', 'C7C6', 'C6C5', 'C5B5', 'B5B4', 'B4B3', 'B3B2', 'B2B1', 'B1A1']
Veículo 1 reroteado pela nova rota ['E9D9', 'D9C9', 'C9B9', 'B9B8', 'B8B7', 'B7B6', 'B6A6', 'A6A5', 'A5A4', 'A4A3', 'A3A2', 'A2A1', 'A1A0']
Veículo 10 reroteado pela nova rota ['E8F8', 'F8F7', 'F7F6', 'F6F5', 'F5F4', 'F4F3', 'F3F2', 'F2F1', 'F1G1', 'G1H1', 'H1I1', 'I1J1', 'J1J0']
Veículo 11 reroteado pela nova rota ['J3J4', 'J4J5', 'J5I5', 'I5I6', 'I6I7', 'I7I8', 'I8I9', 'I9H9', 'H9G9', 'G9F9']
Veículo 12 reroteado pela nova rota ['D0E0', 'E0E1', 'E1E2', 'E2E3', 'E3E4', 'E4E5', 'E5

KeyError: 17

In [ ]:
# Convert results to a DataFrame
df = pd.DataFrame(simulation_results)
df

In [ ]:


plt.figure(figsize=(10, 6))
plt.plot(df['Reroute Interval'], df['Average Time Loss'], marker='o')
plt.title('Average Time Loss vs. Reroute Interval')
plt.xlabel('Reroute Interval (seconds)')
plt.ylabel('Average Time Loss')
plt.grid(True)
plt.show()


In [ ]:
# Display descriptive statistics
df.describe()

In [ ]:
# Specific statistics for each reroute interval
grouped_stats = df.groupby('Reroute Interval').agg(['mean', 'std', 'min', 'max'])
grouped_stats

In [ ]:
# Compute correlation matrix
correlation_matrix = df.corr()

# Display correlation matrix
correlation_matrix

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix Heatmap')
plt.show()


In [ ]:


plt.figure(figsize=(10, 6))
sns.boxplot(x='Reroute Interval', y='Average Time Loss', data=df)
plt.title('Distribution of Time Loss by Reroute Interval')
plt.xlabel('Reroute Interval (seconds)')
plt.ylabel('Average Time Loss')
plt.show()


In [ ]:
# Add a constant term for the intercept in the regression model
X = sm.add_constant(df['Reroute Interval'])
y = df['Average Time Loss']

# Fit the linear regression model
model = sm.OLS(y, X).fit()

# Print the summary of the regression analysis
print(model.summary())


In [ ]:
# Plot waiting time vs reroute interval
plt.figure(figsize=(10, 6))
plt.plot(df['Reroute Interval'], df['Average Waiting Time'], marker='o', label='Average Waiting Time')
plt.plot(df['Reroute Interval'], df['Average Duration'], marker='x', label='Average Duration')
plt.title('Waiting Time and Duration vs. Reroute Interval')
plt.xlabel('Reroute Interval (seconds)')
plt.ylabel('Time (seconds)')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Create a box plot for Average Time Loss vs. Reroute Interval
plt.figure(figsize=(10, 6))
sns.boxplot(x='Reroute Interval', y='Average Time Loss', data=df)
plt.title('Box Plot of Average Time Loss by Reroute Interval')
plt.xlabel('Reroute Interval (seconds)')
plt.ylabel('Average Time Loss')
plt.grid(True)

# Show the plot
plt.show()

In [ ]:
sns.pairplot(df)
plt.suptitle('Pair Plot of Simulation Metrics', y=1.02)
plt.show()